In [1]:
import pandas as pd
from pipeline import Pipeline
from pipeline import build_csv
from stop_words import stop_words
import string
import json
import io
import datetime
import csv

In [2]:
pipeline = Pipeline()

In [3]:
def file_to_json(file):
    with open(file, "r") as f:
        j_file = json.load(f)
    df = pd.json_normalize(j_file['stories'])
    return df

stories =file_to_json("hn_stories_2014.json")

In [4]:
def filter_stories(df):
    df = df[df["points"]>50]
    df = df[df['num_comments']>1]
    df= df[~df['title'].str.startswith('Ask HN')]
    return df
stories = filter_stories(stories)

In [5]:
def json_to_csv(df):
    df = df[['objectID', 'created_at', 'url', 'points', 'title']]
    df.to_csv('cleaned_stories.csv')
    df = pd.read_csv('cleaned_stories.csv', index_col=0, parse_dates=['created_at'])
    return df
stories = json_to_csv(stories)    

In [7]:
#add generator
def extract_titles(df, col):
    title = df['title']
    yield (title)
title = extract_titles(stories, 'title')

In [8]:
def clean_titles(df, col):
#     should not do like htis cause room for more error
#     df[col] = df[col].str.lower()
#     df[col] = df[col].str.replace(f'[{string.punctuation}]', '')
#     df[col] = df[col].str.replace('‘', '').str.replace('’', '')
    df[col] = df[col].str.lower().str.replace(f'[{string.punctuation}]', '').str.replace('‘', '').str.replace('’', '')
    return df
stories = clean_titles(stories, 'title')

In [9]:
def build_keyword_dictionary(df, col):
    counts = {}
    for each in df[col].str.split():
        for word in each:
            if word not in stop_words:
                if word not in counts:
                    counts[word] = 0
                counts[word] += 1
    counts = sorted(counts.items(), key=lambda x: (-x[1], x[0]))
    return counts
word_freq = build_keyword_dictionary(stories, 'title')

In [10]:
def top_100(dictionary):
    dict1 = dictionary[:99]
    tup = tuple(dict1)
    return tup

tup = top_100(word_freq)

In [11]:
tup

(('new', 185),
 ('google', 167),
 ('bitcoin', 101),
 ('open', 92),
 ('programming', 90),
 ('web', 89),
 ('data', 85),
 ('video', 79),
 ('python', 76),
 ('code', 72),
 ('facebook', 71),
 ('released', 71),
 ('using', 71),
 ('2013', 65),
 ('javascript', 65),
 ('free', 64),
 ('source', 64),
 ('game', 63),
 ('internet', 63),
 ('c', 59),
 ('microsoft', 59),
 ('linux', 58),
 ('app', 57),
 ('pdf', 55),
 ('dont', 54),
 ('language', 54),
 ('work', 54),
 ('2014', 52),
 ('software', 52),
 ('startup', 51),
 ('apple', 50),
 ('make', 50),
 ('use', 50),
 ('security', 48),
 ('time', 48),
 ('yc', 48),
 ('github', 45),
 ('nsa', 45),
 ('windows', 44),
 ('1', 41),
 ('like', 41),
 ('way', 41),
 ('world', 41),
 ('computer', 40),
 ('heartbleed', 40),
 ('project', 40),
 ('design', 37),
 ('git', 37),
 ('ios', 37),
 ('users', 37),
 ('ceo', 36),
 ('developer', 36),
 ('life', 36),
 ('os', 36),
 ('twitter', 36),
 ('vs', 36),
 ('big', 35),
 ('day', 35),
 ('android', 34),
 ('online', 34),
 ('court', 33),
 ('simple', 